## 랭체인 통한 LLM활용
- 교재 3~5장은 colab에서 실습

In [18]:
!pip install langchain openai langchain-openai

### open-ai GPT-4o-mini 모델 API 호출
- 상위 키 필드들
  - id :	생성된 대화의 고유 ID. 로깅/추적용
  - object :	객체 타입 → 항상 'chat.completion'
  - created :	생성된 시각 (Unix timestamp)
  - model :	사용된 모델명 (예: "gpt-4o-mini-2024-07-18")
  - service_tier :	사용중인 요금제 (free, default, pro 등)
- choices: 핵심 응답 결과 리스트(예시)   
choices=[Choice(     
  finish_reason='stop',   
  index=0,   
  logprobs=None,   
  message=ChatCompletionMessage(...)
)]
- choices 항목별 설명
  - index : 해당 응답의 번호 (0부터 시작)
  - finish_reason : 응답이 끝난 이유(stop 정상 종료, length 토큰 제한 등)
  - message.role : "assistant" or "user"
  - message.content : 응답 텍스트 본문
  - logprobs : 확률 관련 정보 (사용 안 하면 None)

In [2]:
from openai import OpenAI
from google.colab import userdata

# colab 환경에서는 api key 이름과 key값을 저장할 수 있음
# 교재처럼 실제 key값을 "YOUR_API_KEY"등으로 바꿔서 코드를 저장할 필요 없음
api_key = userdata.get('modulabsKey')  # 비밀 키 불러오기
client = OpenAI(api_key=api_key)

client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = [
    {
        "role": "user",
        "content": "Who won the world series in 2020?"
    }
    ]
)

ChatCompletion(id='chatcmpl-BlUrQn3yESeMFsAeJhgM2VdMB3sjr', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The Los Angeles Dodgers won the World Series in 2020. They defeated the Tampa Bay Rays, clinching the championship in six games. This victory marked the Dodgers' first World Series title since 1988.", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1750659744, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_34a54ae93c', usage=CompletionUsage(completion_tokens=43, prompt_tokens=17, total_tokens=60, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

#### 01 ChatPromptTemplate모듈

##### ChatPromptTemplate예시-1

In [4]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
    # systemMessage: 유용한 챗봇이라는 역할과 이름을 부여
    ("system", "You are a helpful AI bot. Your name is {name}."),
    # HumanMessage와 AIMessage: 서로 안부를 묻고 답하는 대화 히스토리 주입
    ("human","Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    # HumanMessage로 사용자가 입력한 프롬프트를 전달
    ("human","{user_input}"),
    ]

)
messages = chat_template.format_messages(name="Bob", user_input="What is your name?")
print(messages)

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}), HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})]


##### ChatPromptTemplate예시-2

In [6]:
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content = (
                "You are a helpful assistant that re-writes the user's text to sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

message = chat_template.format_messages(text = "I don't like eating tasty things.")
print(messages)

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}), HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})]


#### 02 Temperature따른 대답
- temperature가 낮으면 -> 안정적이고 일관된 대답

In [20]:
# API KEY 저장을 위한 os 라이브러리 호출
import os

# Temperature = 0
chatgpt_temp0_1 = ChatOpenAI(model_name = "gpt-4o-mini", temperature = 0)
chatgpt_temp0_2 = ChatOpenAI(model_name = "gpt-4o-mini", temperature = 0)

# Temperature = 1
chatgpt_temp1_1 = ChatOpenAI(model_name="gpt-4o-mini", temperature = 1)
chatgpt_temp1_2 = ChatOpenAI(model_name="gpt-4o-mini", temperature = 1)

model_list = [chatgpt_temp0_1,chatgpt_temp0_2,chatgpt_temp1_1,chatgpt_temp1_2]

for i in model_list:
  answer = i.invoke("왜 파이썬이 가장 인기 있는 프로그래밍 언어인지 한 문장으로 설명해줘",max_tokens=128)
  print("-"*100)
  print(">>>",answer.content)

----------------------------------------------------------------------------------------------------
>>> 파이썬은 간결하고 읽기 쉬운 문법, 다양한 라이브러리와 프레임워크, 그리고 광범위한 커뮤니티 지원 덕분에 데이터 과학, 웹 개발, 인공지능 등 다양한 분야에서 널리 사용되기 때문에 가장 인기 있는 프로그래밍 언어입니다.
----------------------------------------------------------------------------------------------------
>>> 파이썬은 간결하고 읽기 쉬운 문법, 다양한 라이브러리와 프레임워크, 그리고 광범위한 커뮤니티 지원 덕분에 데이터 과학, 웹 개발, 인공지능 등 다양한 분야에서 널리 사용되기 때문에 가장 인기 있는 프로그래밍 언어입니다.
----------------------------------------------------------------------------------------------------
>>> 파이썬은 간결하고 읽기 쉬운 문법 덕분에 다양한 분야에서 빠르게 배우고 사용할 수 있어, 데이터 과학, 웹 개발, 인공지능 등 많은 분야에서 인기를 끌고 있습니다.
----------------------------------------------------------------------------------------------------
>>> 파이썬은 간결하고 직관적인 문법으로 학습이 쉬우며, 광범위한 라이브러리와 프레임워크 덕분에 데이터 과학, 웹 개발, 자동화 등 다양한 분야에 효과적으로 활용할 수 있기 때문에 가장 인기 있는 프로그래밍 언어입니다.


##### 교재와 다른 예시
- temperature 0.3, 0.5, 0.7, 0.9로 설정
- 질문 : "반도체 8대 공정에 대해 각 공정별로 한 문장씩으로 설명해줘"

In [22]:
import os

# model_list

model_temp_1 = ChatOpenAI(model_name = "gpt-4o-mini", temperature=0.3)
model_temp_2 = ChatOpenAI(model_name = "gpt-4o-mini", temperature=0.5)
model_temp_3 = ChatOpenAI(model_name = "gpt-4o-mini", temperature=0.7)
model_temp_4 = ChatOpenAI(model_name = "gpt-4o-mini", temperature=0.9)

model_list = [model_temp_1, model_temp_2, model_temp_3, model_temp_4]

for i in model_list:
  answer = i.invoke("반도체 8대 공정에 대해 각 공정별로 한 문장씩 요약해서 설명해줘.", max_tokens=2000)
  print("-"*100)
  print(">>>",answer.content)


----------------------------------------------------------------------------------------------------
>>> 반도체 제조 공정은 여러 단계로 나뉘며, 일반적으로 8대 공정으로 요약할 수 있습니다. 각 공정별로 간단히 설명하겠습니다.

1. **산화(Oxidation)**: 실리콘 웨이퍼 표면에 산화막을 형성하여 전기적 특성을 조절하고, 불순물로부터 보호합니다.

2. **포토리소그래피(Photolithography)**: 감광제를 사용하여 웨이퍼에 패턴을 전사하고, 이후 이 패턴을 기반으로 다양한 구조를 형성합니다.

3. **식각(Etching)**: 포토리소그래피로 형성된 패턴을 따라 웨이퍼의 특정 부분을 제거하여 원하는 형태를 만듭니다.

4. **이온 주입(Ion Implantation)**: 특정 불순물을 웨이퍼에 주입하여 전도성을 조절하고, 반도체의 전기적 특성을 개선합니다.

5. **증착(Deposition)**: 화학적 또는 물리적 방법을 통해 웨이퍼 표면에 얇은 막을 형성하여 다양한 기능을 추가합니다.

6. **열처리(Annealing)**: 웨이퍼를 고온에서 처리하여 불순물의 확산을 촉진하고, 결정 구조를 개선하여 전기적 특성을 최적화합니다.

7. **패키징(Packaging)**: 완성된 칩을 보호하고 외부와 연결할 수 있도록 패키지에 장착하여 최종 제품으로 만듭니다.

8. **테스트(Test)**: 제조된 반도체 칩의 성능과 신뢰성을 확인하기 위해 전기적 특성을 검사합니다.

이러한 공정을 통해 반도체 소자가 제작되며, 각 단계는 제품의 품질과 성능에 중요한 영향을 미칩니다.
----------------------------------------------------------------------------------------------------
>>> 반도체 제조 과정은 여러 단계로 나뉘며, 주요 8대 공정은 다음과 같습니다:

1. **웨이퍼 

#### 03 답변 스트리밍하기
- LLM이 실제로 작동하는 방식은 답변을 한 단어씩 생성하기 때문에, 모델의 작동과정을   
실시간으로 화면에 표시해주는데 이것을 스트리밍(streaming)이라고 지칭

In [24]:
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(model_name="gpt-4o-mini", temperature = 0)
for chunk in chat.stream("달에 관한 시를 써줘"):
  print(chunk.content, end="", flush=True)

**달빛의 노래**

밤하늘에 떠오른 달,  
은빛의 눈물처럼 빛나네.  
조용한 바람에 실려오는  
그대의 속삭임, 나를 감싸네.

구름 사이로 얼굴을 내밀고,  
어둠 속에 길을 비추는  
그대의 따스한 미소는  
잃어버린 꿈을 다시 일깨우네.

고요한 호수에 비친 모습,  
영원히 변치 않을 것 같은  
그대의 존재는 나의 마음,  
밤마다 그리움으로 가득 차네.

달이여, 나의 친구여,  
이 밤도 함께해 주오.  
별들과 함께 춤추며,  
우리의 이야기를 나누자, 영원히.

In [25]:
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.5)
for chunk in chat.stream("타지마할에 관한 시를 써줘. 샤자한의 뭄타즈 마할에 대한 마음을 담아서."):
  print(chunk.content, end="", flush = True)

타지마할, 흰 대리석의 꿈  
샤자한의 사랑, 뭄타즈를 향한 그리움  
밤하늘의 별처럼 빛나는 그녀의 미소  
세상의 모든 아름다움, 그녀에게 바쳐진 성전

강가에 서서, 물결에 비친 모습  
그녀의 영혼이 이곳에 깃들어  
시간이 멈춘 듯, 영원히 함께할  
사랑의 기념비, 찬란한 이 순간

눈부신 햇살 아래, 조각된 사랑  
각기 다른 색으로 물드는 저녁 노을  
가슴 속 깊이 새겨진 그 이름  
뭄타즈, 나의 영원한 여인

슬픔과 기쁨이 얽혀 있는 이곳  
사랑의 비극, 그러나 끝없는 희망  
타지마할, 너는 나의 마음  
영원히 함께할 우리의 이야기

세월이 흐르고, 세상이 변해도  
너를 향한 내 사랑은 변치 않아  
타지마할, 너는 나의 기도  
영원히 간직할, 뭄타즈의 기억.

#### 04 응답 캐싱하기
- 사용자 질문과 이에 대한 LLM의 답변을 메모리에 쌓아두면, 이 메모리가 쌓일수록   
사용자의 질문에 대한 응답 시간을 줄일 가능성이 높아짐

In [34]:
# 교재 작성 시점과 현재의 langchain-community간에 괴리가 있어서 버전 맞춰야 함!
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.65
    Uninstalling langchain-core-0.3.65:
      Successfully uninstalled langchain-core-0.3.65
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.25
    Uninstalling langchain-0.3.25:
      Successfully uninstalled langchain-0.3.25


In [2]:
from openai import OpenAI
from google.colab import userdata
import os

api_key = userdata.get("modulabsKey")  # Colab 비밀키 불러오기
os.environ["OPENAI_API_KEY"] = api_key  # 환경변수에 등록!
client = OpenAI(api_key=api_key)

In [3]:
# [1] 캐시 설정 및 모델 준비
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI

set_llm_cache(InMemoryCache())
chat = ChatOpenAI(model_name = "gpt-4o-mini", temperature = 0)

In [4]:
# [2] 실제 응답 + 시간 측정
%%time
chat.invoke("일반 상대성 이론을 한마디로 설명해줘")

CPU times: user 26.1 ms, sys: 1.97 ms, total: 28.1 ms
Wall time: 1.82 s


AIMessage(content='일반 상대성 이론은 중력이 시공간의 곡률로 설명되며, 질량이 큰 물체가 주변의 시공간을 휘게 만들어 다른 물체들이 그 곡률을 따라 움직인다는 이론입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 21, 'total_tokens': 76, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BlUvLrYBBp9MTDoXSgfn56V2BIAPL', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--ddb9e238-3521-438e-a5aa-8a933425522c-0', usage_metadata={'input_tokens': 21, 'output_tokens': 55, 'total_tokens': 76, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [5]:
# 같은 질문 다시해보기
%%time
chat.invoke("일반 상대성 이론을 한마디로 설명해줘")

CPU times: user 581 µs, sys: 0 ns, total: 581 µs
Wall time: 577 µs


AIMessage(content='일반 상대성 이론은 중력이 시공간의 곡률로 설명되며, 질량이 큰 물체가 주변의 시공간을 휘게 만들어 다른 물체들이 그 곡률을 따라 움직인다는 이론입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 21, 'total_tokens': 76, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BlUvLrYBBp9MTDoXSgfn56V2BIAPL', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--ddb9e238-3521-438e-a5aa-8a933425522c-0', usage_metadata={'input_tokens': 21, 'output_tokens': 55, 'total_tokens': 76, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

#### 실습하기
- 스트리밍되는 AI스터디 플래너 챗봇 만들기
  - 모델 호출
  - ChatPromptTemplate으로 SystemMessage, HumanMessage 프롬프트 설정
  - stream()함수를 통한 답변 스트리밍

In [8]:
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI

# GPT-4o-mini 모델 호출
chat = ChatOpenAI(model_name = "gpt-4o-mini", temperature = 0)

# ChatPromptTemplate 통해 스터디 플래너 역할 부여 및 사용자 프롬프트 매개변수화
chat_template = ChatPromptTemplate.from_messages(
    [
    SystemMessage(
        content = (
            "당신은 공부 계획을 세워주는 스터디 플래너 머신입니다."
            "사용자의 공부 주제를 입력받으면, 이를 학습하기 위한 공부 계획을 작성합니다."
        )

    ),
    HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

# 앞서 설정한 프롬프트 템플릿에 HumanMessage로 문장 전달
messages = chat_template.format_messages(text="Large Language Model에 대해서 공부하고 싶어요.")

# stream 함수를 통해 답변 스트리밍

for chunk in chat.stream(messages):
  print(chunk.content, end="", flush = True)

Large Language Model(LLM)에 대한 공부 계획을 세워드리겠습니다. 이 계획은 4주 동안 진행되며, 매주 주제를 나누어 학습할 수 있도록 구성하였습니다.

### 1주차: LLM의 기초 이해
- **목표**: LLM의 기본 개념과 역사 이해
- **학습 내용**:
  - LLM의 정의 및 작동 원리
  - 자연어 처리(NLP)의 기초
  - LLM의 발전 역사 (예: GPT, BERT 등)
- **추천 자료**:
  - 온라인 강의: Coursera, edX의 NLP 기초 강의
  - 관련 논문: "Attention is All You Need" (Transformer 모델 소개)
- **활동**:
  - LLM 관련 블로그 글 읽기 (예: OpenAI 블로그)
  - LLM의 발전 과정 정리하기

### 2주차: LLM의 구조와 기술
- **목표**: LLM의 구조와 기술적 요소 이해
- **학습 내용**:
  - Transformer 아키텍처의 구성 요소 (Self-Attention, Positional Encoding 등)
  - Pre-training과 Fine-tuning의 개념
  - LLM의 학습 과정 (데이터 수집, 모델 훈련)
- **추천 자료**:
  - "The Illustrated Transformer" 블로그 포스트
  - 관련 유튜브 강의
- **활동**:
  - Transformer 구조를 시각적으로 정리하기
  - Pre-training과 Fine-tuning의 차이점 정리하기

### 3주차: LLM의 응용
- **목표**: LLM의 다양한 응용 사례 이해
- **학습 내용**:
  - LLM을 활용한 텍스트 생성, 번역, 요약 등
  - LLM의 실제 사용 사례 (예: ChatGPT, AI 작문 도구)
  - LLM의 한계와 윤리적 고려사항
- **추천 자료**:
  - 사례 연구: LLM을 활용한 기업 사례
  - 관련 논문: LLM의 윤리적 문제에 대한 논의
- **활동**:
  - LLM을 활용한 프로젝트 